**Applying Machine Learning Algorithms**

*Importing Necessary Libraries*

In [1]:
# IMPORTING NECESSARY LIBRARIES
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


*Loading Dataset*

In [2]:
'''
survival - Survival (0 = No; 1 = Yes)
class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
name - Name
sex - Sex
age - Age
sibsp - Number of Siblings/Spouses Aboard
parch - Number of Parents/Children Aboard
ticket - Ticket Number
fare - Passenger Fare
cabin - Cabin
embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat - Lifeboat (if survived)
body - Body number (if did not survive and body was recovered)
'''

'\nsurvival - Survival (0 = No; 1 = Yes)\nclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)\nname - Name\nsex - Sex\nage - Age\nsibsp - Number of Siblings/Spouses Aboard\nparch - Number of Parents/Children Aboard\nticket - Ticket Number\nfare - Passenger Fare\ncabin - Cabin\nembarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)\nboat - Lifeboat (if survived)\nbody - Body number (if did not survive and body was recovered)\n'

In [3]:
# Training Dataset
train_df = pd.read_csv('https://raw.githubusercontent.com/Revati-N/AI-DS-Projects/main/Titanic/Datasets/train.csv')
train_df.Name = 'Training Set'


In [4]:
# Data Processing
train_df['Sex'] = train_df['Sex'].map({'female': 1, 'male': 0})
train_df['Embarked'] = train_df['Embarked'].map({'C': 0, 'S': 1, 'Q':2})

In [5]:
# Testing Dataset
test_df = pd.read_csv('https://raw.githubusercontent.com/Revati-N/AI-DS-Projects/main/Titanic/Datasets/test.csv')
test_df.Name = 'Testing Set'

In [6]:
test_df['Sex'] = test_df['Sex'].map({'female': 1, 'male': 0})
test_df['Embarked'] = test_df['Embarked'].map({'C': 0, 'S': 1, 'Q':2})

*Pre-Processing*

In [7]:
train_df.info() #check for null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


In [9]:
def null_value(df):
    percentage = ((df.isna().sum()/df.isna().count())*100).sort_values(ascending = False)
    count = (df.isna().sum()).sort_values(ascending = False)
    diff= pd.concat([count,percentage],axis = 1, keys = ["Count", "Percentage"])
    return diff

In [10]:
null_value(train_df)

,Count,Percentage
Cabin,687,77.104377
Age,177,19.865320
Embarked,2,0.224467
PassengerId,0,0.000000
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000


In [11]:
null_value(test_df)

,Count,Percentage
Cabin,327,78.229665
Age,86,20.574163
Fare,1,0.239234
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000


In [12]:
#fill up or replace the null in the age with the mean of age
train_df["Age"].replace(np.nan, train_df["Age"].mean(), inplace = True)
train_df["Embarked"].replace(np.nan, train_df["Embarked"].mode()[0], inplace = True)
train_df["Age"].isna().sum(), train_df["Embarked"].isna().sum()

(0, 0)

In [13]:
test_df["Age"].replace(np.nan, test_df["Age"].mean(),inplace = True)
test_df["Fare"].replace(np.nan, test_df["Fare"].mode()[0],inplace = True)
test_df["Age"].isna().sum(), test_df["Fare"].isna().sum()

(0, 0)

In [14]:
#Dropping of the Name and the passenger ticket because both has no recurring pattern
train_df.drop(["Name","Ticket"], axis =1, inplace = True)
test_df.drop(["Name","Ticket"],axis = 1, inplace = True)

In [15]:
#combining the SibSp and Parch
def merge(df):
    rel = []
    for i in df["SibSp"].values.tolist():
        rel.append(i)
    rel1 = []
    for i in df["Parch"].values.tolist():
        rel1.append(i)
        
    concat = []
    for index in range(len(rel)):
        concat.append(rel[index] + rel1[index])
        
    df1 = pd.DataFrame(concat, columns = ["Relatives"])
    df = pd.concat([df,df1], axis = 1)
    return df

In [16]:
train = merge(train_df)

In [17]:
train.drop(["SibSp","Parch"],axis=1, inplace = True)

In [18]:
train.loc[517]

PassengerId          518
Survived               0
Pclass                 3
Sex                    0
Age            29.699118
Fare               24.15
Cabin                NaN
Embarked             2.0
Relatives              0
Name: 517, dtype: object

In [19]:
test = merge(test_df)
test.drop(["SibSp","Parch"],axis = 1, inplace = True)

In [20]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,Relatives
0,892,3,0,34.5,7.8292,NaN,2,0
1,893,3,1,47.0,7.0000,NaN,1,1
2,894,2,0,62.0,9.6875,NaN,2,0
3,895,3,0,27.0,8.6625,NaN,1,0
4,896,3,1,22.0,12.2875,NaN,1,2


*Feature Selection*

In [21]:
# Importing Libraries
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [22]:
train.drop("PassengerId",axis = 1, inplace=True)
test.drop({"PassengerId","Cabin"},axis = 1, inplace=True)

In [23]:
x = train.drop({"Survived","Cabin"},axis = 1)
y = train["Survived"]

In [24]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 10)
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)
test = StandardScaler().fit_transform(test)

**Application of Machine Learning Algorithms**

*Gaussian Naive Bayes*

In [25]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train,y_train)

y_pred = gnb.predict(X_test)

NBAccuracy = accuracy_score(y_test, y_pred)
print("Gaussian Naive Bayes Accuracy: ",NBAccuracy)


Gaussian Naive Bayes Accuracy:  0.7932960893854749
